In [ ]:
from yfapi import YahooFinanceAPI, Interval
from datetime import datetime, timedelta
import psycopg2
from sqlalchemy import create_engine
import os

def lambda_handler(event, context):
    
    # Define database credentials and path
    dbuname = os.environ['DBUNAME']
    dbpwd = os.environ['DBPWD']
    host = os.environ['HOST']
    port = os.environ['PORT']
    dbname = os.environ['DBNAME']
    
    # # instantiate the API and set the interval to daily data
    dh = YahooFinanceAPI(Interval.DAILY)
            
    # Define function for fetching and loading the data
    def fetch_load_data(instrument, start, end, method):
   
        while True:    
            #fetch data
            try:
                df = dh.get_ticker_data(instrument, start, end)
                table_name = 'table_yf_'+labels[instrument]
                df['Ins'] = instrument
                df['Ins_label'] = labels[instrument]
            except Exception as e:
                print(e)
                print("Investment Instrument not available: " + instrument)
                break
       
               
            #load data
            if_ex_val = method
            
            try:
                conn_string = 'postgresql://'+dbuname+':'+dbpwd+'@'+host+':'+port+'/'+dbname
                engine = create_engine(conn_string)
                df.to_sql(table_name, conn_string, if_exists = if_ex_val, index=False) 
            except Exception as e:
                print(e)
                print("Data load failed: " + table_name)   
                break
            break    
   
    # Define investment assets
    inv_instruments = ['URTH', 'EXW1.DE', 'CSSMI.SW', 'EXXT.DE', 'GC=F', 'SI=F']
    labels={'GC=F':'gold','SI=F':'silver','URTH':'msci_world', 'EXW1.DE':'euro_stoxx', 'CSSMI.SW':'smi', 'EXXT.DE':'nasdaq' }

    # Daily load of new data

    # get datetime of yesterday at 00:00:00
    yesterday = datetime.strftime(datetime.today() - timedelta(1), "%Y, %m, %d")
    today = datetime.strftime(datetime.today(), "%Y, %m, %d")           
    
    # set timeframe
    now = datetime.strptime(today, "%Y, %m, %d")
    then = datetime.strptime(yesterday, "%Y, %m, %d")
   
    # set method of function df.to_sql()
    method = 'append'
   
    # execute function
    for ins in inv_instruments:
        fetch_load_data(ins, then, now, method)        
    